In [1]:
# %pip install -q sentencepiece
# %pip install -q protobuf
# %pip install -q -U bitsandbytes
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git
# %pip install -q scipy
# %pip install -q -U trl

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel, PeftConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset, concatenate_datasets
import pandas as pd
import math

/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
from torch import nn
import re
import ast
import numpy as np

class CustomSFTTrainer(SFTTrainer):
    def __init__(self, tokenizer, *args, **kwargs):
        super(CustomSFTTrainer, self).__init__(*args, **kwargs)
        self.tokenizer = tokenizer

    def compute_loss(self, model, inputs, return_outputs=False):
        inputs_copy = inputs.copy()
        labels = inputs_copy.pop("labels")
        outputs = model(**inputs_copy)
        logits = outputs.logits
        
        # Compute the loss using the superclass's compute_loss method
        super_loss, _ = super().compute_loss(model, inputs, return_outputs=True)
        print("\npreloss:", super_loss.item())
        loss = super_loss
        
        wrong_proportion = 1
        
        labels_decoded = decoded = self.tokenizer.batch_decode(labels.tolist())
        labels_result = labels_decoded[0]
        
        try:
            target_match = re.search(r"Test_1_Output=\[\[.*?\]\]", labels_result)
            target_array_string = target_match.group(0).replace("Test_1_Output=", "")
            target_array = np.array(ast.literal_eval(target_array_string))
            
            # Apply softmax to logits to get probabilities
            probs = torch.softmax(logits, dim=-1)
            # Get the predicted tokens (highest probability)
            _, predictions = torch.max(probs, dim=-1)
            
            decoded = self.tokenizer.batch_decode(predictions.tolist())
            predictions_result = decoded[0]
            match1 = re.search(r"Test_1_Output=\[\[.*?\]\]", predictions_result)
            predicted_array_string = ""
            if match1:
                predicted_array_string = match1.group(0).replace("Test_1_Output=", "")
                print("matched Test_1_Output=[[*?]]")
            else:
                loss *= 2
                print("Did not match Test_1_Output=[[*?]]..predicted")
                print(predictions_result)

            if predicted_array_string != "":
                try:
                    predicted_array = np.array(ast.literal_eval(predicted_array_string))
                    if predicted_array.shape == target_array.shape:
                        print("correct shape")
                        loss *= loss
                        equal_elements_count = np.sum(np.equal(target_array, predicted_array))
                        total_elements_count = target_array.size + 1
                        wrong_elements_count = total_elements_count - equal_elements_count
                        wrong_proportion = wrong_elements_count / total_elements_count
                        loss *= wrong_proportion * wrong_proportion
                        are_equal = np.array_equal(predicted_array, target_array)
                        if are_equal:
                            print("are equal!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                            print("target")
                            print(target_array)
                            print("predicted")
                            print(predicted_array)
                            loss *= loss
                    else:
                        print("incorrect shape")
                        print("target")
                        print(target_array)
                        print("predicted")
                        print(predicted_array)
                except:
                    print("failed to make predicted array")
                    print(predicted_array_string)
        except:
            print("failed to make target array")
            print(labels_result)
        
        print("wp:", wrong_proportion, "  loss:", loss.item())

        return (loss, outputs) if return_outputs else loss

In [4]:
def generate_prompt(data_point):
    user_message = data_point["instruction"]
    assistant_message = data_point["output"]
    text = f"<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n{assistant_message}<|im_end|>"
    return text

In [5]:
base_model_name = "merged_models/Orca_2_finetune_04_merge"
new_adapter_name = "Orca-2-13b-4bit-finetune-on-ARC-15"

# Load the entire model on the GPU 0
device_map = {"": 0}

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [6]:
# Load the data using pandas
data_file = "data/core_ARC_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset3 = Dataset.from_pandas(df)

# Load the data using pandas
data_file = "data/ARC_augmented_training_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset1 = Dataset.from_pandas(df)

# Load the data using pandas
data_file = "data/ARC_augmented_evaluation_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset2 = Dataset.from_pandas(df)

# Generate the "prompt" column for each dataset
dataset1 = dataset1.map(lambda data_point: {"prompt": generate_prompt(data_point)})
dataset2 = dataset2.map(lambda data_point: {"prompt": generate_prompt(data_point)})
dataset3 = dataset3.map(lambda data_point: {"prompt": generate_prompt(data_point)})

# # Assuming dataset1 and dataset2 are instances of Dataset from the `datasets` library
dataset = concatenate_datasets([dataset1, dataset2, dataset3])

Map:   0%|          | 0/72504 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11240 [00:00<?, ? examples/s]

In [7]:
def main():
    # Load base model
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map=device_map,
    )
    model.config.use_cache = False
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)  

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, add_eos_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

    per_device_train_batch_size = 1
    gradient_accumulation_steps = 1
    max_seq_length = 5000

    output_dir = "results/" + new_adapter_name

    steps_per_epoch = len(dataset) // (
        per_device_train_batch_size * gradient_accumulation_steps
    )
    print("Steps:", steps_per_epoch)

    # Set training parameters
    training_arguments = TrainingArguments(
        output_dir=output_dir,
        max_steps=steps_per_epoch,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        optim="paged_adamw_32bit",
        lr_scheduler_type="cosine",
        weight_decay=0.01
        save_strategy="steps",
        evaluation_strategy="no",
        save_steps=1000,
        logging_steps=1,
        learning_rate=5e-6,
        fp16=True,
        warmup_steps=0.03,
        group_by_length=True,
        gradient_checkpointing=True,
    )

    trainer = CustomSFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        dataset_text_field="prompt",
        max_seq_length=max_seq_length,
        tokenizer=tokenizer,
        args=training_arguments,
    )

    # Train model
    trainer.train()

    # Save trained model
    adapter = "adapters/" + new_adapter_name
    trainer.model.save_pretrained(adapter)

In [8]:
main()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Steps: 93744


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/93744 [00:00<?, ? examples/s]

...


preloss: 0.1961781233549118
matched Test_1_Output=[[*?]]
wp: 1   loss: 0.1961781233549118

preloss: 0.18428097665309906
matched Test_1_Output=[[*?]]
correct shape
wp: 0.09900990099009901   loss: 0.001824564184062183

preloss: 0.30913543701171875
matched Test_1_Output=[[*?]]
correct shape
wp: 0.21782178217821782   loss: 0.006733643356710672

preloss: 0.1427215337753296
matched Test_1_Output=[[*?]]
correct shape
wp: 0.06593406593406594   loss: 0.0009410211350768805

preloss: 0.17263314127922058
matched Test_1_Output=[[*?]]
correct shape
wp: 0.06   loss: 0.0010357988066971302

preloss: 0.08762305974960327
matched Test_1_Output=[[*?]]
correct shape
wp: 0.039603960396039604   loss: 0.0003470220253802836

preloss: 0.12177947908639908
matched Test_1_Output=[[*?]]
correct shape
wp: 0.06097560975609756   loss: 0.0007425578078255057

preloss: 0.19514629244804382
matched Test_1_Output=[[*?]]
correct shape
wp: 0.1592920353982301   loss: 0.0031085251830518246

preloss: 0.2966654300689697
matched T


preloss: 0.3757888376712799
matched Test_1_Output=[[*?]]
correct shape
wp: 0.5   loss: 0.018789442256093025

preloss: 0.2959384322166443
matched Test_1_Output=[[*?]]
correct shape
wp: 0.6   loss: 0.017756307497620583

preloss: 0.32712453603744507
matched Test_1_Output=[[*?]]
correct shape
wp: 0.3   loss: 0.009813737124204636

preloss: 0.0806250274181366
matched Test_1_Output=[[*?]]
wp: 1   loss: 0.0806250274181366

preloss: 0.3366393744945526
wp: 1   loss: 3.366393804550171

preloss: 0.2996487021446228
wp: 1   loss: 2.9964871406555176

preloss: 0.1192750632762909
matched Test_1_Output=[[*?]]
correct shape
wp: 0.8   loss: 0.009542006067931652

preloss: 0.060793641954660416
matched Test_1_Output=[[*?]]
wp: 1   loss: 0.060793641954660416

preloss: 0.2618654668331146
matched Test_1_Output=[[*?]]
wp: 1   loss: 0.2618654668331146

preloss: 0.2658790647983551
matched Test_1_Output=[[*?]]
correct shape
wp: 0.0546448087431694   loss: 0.0014528910396620631

preloss: 0.08114343881607056
matched 

/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(



preloss: 0.20243214070796967
wp: 1   loss: 2.0243213176727295

preloss: 0.13038411736488342
matched Test_1_Output=[[*?]]
wp: 1   loss: 0.13038411736488342

preloss: 0.18335263431072235
wp: 1   loss: 1.833526372909546

preloss: 0.0964556336402893
wp: 1   loss: 0.9645563364028931

preloss: 0.13797806203365326
matched Test_1_Output=[[*?]]
correct shape
are equal!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
wp: 0.058823529411764705   loss: 8.116357435028476e-07

preloss: 0.1867261826992035
matched Test_1_Output=[[*?]]
wp: 1   loss: 0.1867261826992035

preloss: 0.09697873890399933
matched Test_1_Output=[[*?]]
correct shape
wp: 0.048672566371681415   loss: 0.0004720204451587051

preloss: 0.0623861663043499
matched Test_1_Output=[[*?]]
wp: 1   loss: 0.0623861663043499

preloss: 0.06758147478103638
matched Test_1_Output=[[*?]]
correct shape
wp: 0.027237354085603113   loss: 0.00018407405877951533

preloss: 0.15318241715431213
matched Test_1_Output=[[*?]]
wp: 1   loss: 0.15318241715431

KeyboardInterrupt: 